In [28]:
#DATA AUGMENTATION

# Importing necessary functions
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os

def augment_images(image_path, medicine):
		# Loading a sample image
		img = load_img(image_path)
		# Converting the input sample image to an array
		x = img_to_array(img)
		# Reshaping the input image
		x = x.reshape((1, ) + x.shape)

		# Generating and saving 5 augmented samples
		# using the above defined parameters.
		i = 0
		for batch in datagen.flow(x, batch_size = 16,
								save_to_dir ='C:/Users/Camille/Downloads/NEW PATH', #Camille
								save_prefix =medicine, save_format ='jpg'):
			i += 1
			if i > 5:
				break

# Initialising the ImageDataGenerator class.
# We will pass in the augmentation parameters in the constructor.
datagen = ImageDataGenerator(
		rotation_range = 5,
		shear_range = 1,
		zoom_range = 0.2,
		rescale=1)

input_path = 'C:/Users/Camille/Downloads/CROPPED LABELED RESIZED PRES PICS' #Camille
for filename in os.listdir(input_path):
	if filename.endswith(".jpg"):
		if filename.find("Azathioprine") != -1:
			image_path = os.path.join(input_path, filename)
			medicine = "Azathioprine"
			augment_images(image_path, medicine)
		elif filename.find("Ceftriaxone") != -1:
			image_path = os.path.join(input_path, filename)
			medicine = "Ceftriaxone"
			augment_images(image_path, medicine)
		elif filename.find("Chlorpromazine") != -1:
			image_path = os.path.join(input_path, filename)
			medicine = "Chlorpromazine"
			augment_images(image_path, medicine)
		elif filename.find("Dobutamine") != -1:
			image_path = os.path.join(input_path, filename)
			medicine = "Dobutamine"
			augment_images(image_path, medicine)
		elif filename.find("Hydroxyzine") != -1:
			image_path = os.path.join(input_path, filename)
			medicine = "Hydroxyzine"
			augment_images(image_path, medicine)
		elif filename.find("Lorazepam") != -1:
			image_path = os.path.join(input_path, filename)
			medicine = "Lorazepam"
			augment_images(image_path, medicine)
		elif filename.find("Metronidazole") != -1:
			image_path = os.path.join(input_path, filename)
			medicine = "Metronidazole"
			augment_images(image_path, medicine)
		elif filename.find("Prednisolone") != -1:
			image_path = os.path.join(input_path, filename)
			medicine = "Prednisolone"
			augment_images(image_path, medicine)
		elif filename.find("Quinine") != -1:
			image_path = os.path.join(input_path, filename)
			medicine = "Quinine"
			augment_images(image_path, medicine)
		elif filename.find("Risperidone") != -1:
			image_path = os.path.join(input_path, filename)
			medicine = "Risperidone"
			augment_images(image_path, medicine)
		elif filename.find("Rituximab") != -1:
			image_path = os.path.join(input_path, filename)
			medicine = "Rituximab"
			augment_images(image_path, medicine)
		else:
			image_path = os.path.join(input_path, filename)
			medicine = "Tramadol"
			augment_images(image_path, medicine)

In [29]:
# # Creating Train / Val / Test folders (One time use)
import os
import numpy as np
import shutil
import random
root_dir = 'C:/Users/Camille/Downloads/splitted data/' #Camille

val_ratio = 0
test_ratio = 0.3

os.makedirs(root_dir +'train/')
os.makedirs(root_dir +'test/')

# Creating partitions of the data after shuffeling
src = 'C:/Users/Camille/Downloads/NEW PATH' # Folder to copy images from --Camille

allFileNames = os.listdir(src)
np.random.shuffle(allFileNames)
train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                          [int(len(allFileNames)* (1 - (val_ratio + test_ratio))), 
                                                           int(len(allFileNames)* (1 - test_ratio))])


train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]

print('Total images: ', len(allFileNames))
print('Training: ', len(train_FileNames))
#print('Validation: ', len(val_FileNames))
print('Testing: ', len(test_FileNames))

# Copy-pasting images
for name in train_FileNames:
    shutil.copy(name, root_dir +'train/' )

#for name in val_FileNames:
 #   shutil.copy(name, root_dir +'val/' + cls)

for name in test_FileNames:
    shutil.copy(name, root_dir +'test/' )

Total images:  1575
Training:  1102
Testing:  473


In [6]:
#RGB TO GREYSCALE CAMILLE
import cv2

def greyscale(image):
    grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return grey

In [ ]:
#BINARIZATION TRIX

In [ ]:
#NOISE REMOVAL ASH

In [ ]:
#CED CARL

In [ ]:
#SKELETONIZATION ALL

In [ ]:
#PREPROCESSING IMAGES